In [49]:
import csv # for writing dataframes to csv
import random # for making a random choice
import os # for scanning directories
import itertools
import string # for generating strings
from collections import Counter

import kintypes as kt # bringing large lists of kin types into the namespace
import math # for calculating logs
import pandas as pd

testing = True # set to True to run code blocks with tests and examples

# Internal co-selection

Internal co-selection refers to the tendency for kinship systems to have cross-generational consistency in the terminological distinctions or mergers that are made. That is, if your parents' elder brothers share a kin term, then so too will their children. Or if your parents' sisters are distinguished from your parents' brothers, so too will their children be distinguished. 

Imagine a kinship system like so: as in English, you call your parents' brothers are  *uncles*, and their sisters *aunts*. You call the child of your uncle a *chuncle*, and the child of your aunt a *chaunt*. Thus, you make the same sorts of distinctions among your parents' siblings' generation of kin as are made among your own generation of kin - and you can be certain about which children belong to which parents as a result. This is an example of internal co-selection.

In this notebook, we will gather information about the robustness of this tendency cross-linguistically, using data from Kinbank, a global database of kin terminology. We will also create simulations of existing kinship systems to find out whether internal co-selection is more common in kinship systems cross-linguistically (for a given amount of terminological variation) than we would expect by chance.

We will measure internal co-selection in terms of the **mutual information** between Generation N and Generation N+1 in a particular kinship system. That tells us how much information can be gained from one generation by observing the other - how certain we can be about which children 'go with' which parents. This can be calculated as the **entropy** of one generation (how much unpredictable variation there is) minus the **conditional entropy** between the two generations (how much unpredictability remains in one generation after observing another).

## The procedure

To calculate the mutual information (MI) of a particular kinship system, we must perform the following steps:

1. Extract kin terminology data from Kinbank for this language.
2. Condense the full kinship system down to the terms we are interested in: Ego's generation and Ego's parents' generation.
3. Calculate the probabilities of each kin term within the generation in which it belongs; and the probabilities of each parent-child pair.
4. Calculate entropy, conditional entropy, subtract them from each other to get the mutual information of the system.

After we get that going, we can do these same calculations on simulated kinship systems.

### Extract kin terminology from Kinbank

First, let's actually load our data in. The following function `get_kb_files()` pulls the full list of Kinbank filenames. Later, we can iterate through these to generate MI values for every language in our dataset.

In [3]:
def get_kb_files() -> list:
    files = []
    path = '../languages/kinbank'
    directory = os.scandir(path)
    for file in directory:
        files.append(file.name)
    return files

all_kb_files = get_kb_files()

Using one of these filenames, we can extract the kin terminology from that file and populate a dictionary with it. We're only interested in two columns from the Kinbank data: `parameter`, which contains a short code indicating a **kin type**, and `word`, which contains the **kin term** associated with that kin type. An example of a row in the English data would be `mMeB, uncle`, where `mMeB` means 'male speaker's mother's older brother', and `uncle` is the term associated with that person.

In [2]:
def get_kin_terms(filepath: str) -> dict:
    kin_system = {}
    with open(filepath, encoding='utf8') as f:
        csv_reader = csv.DictReader(f)
        next(csv_reader) # to skip the header row
        for line in csv_reader:
            kin_type = line['parameter']
            kin_term = line['word']
            kin_system[kin_type] = kin_term
    return kin_system

Let's pick a random kinship system to test with throughout this notebook.

In [4]:
if testing:
    
    random.seed(47) # set a seed for reproducibility

    file = random.choice(all_kb_files) # pick a random filename from all_kb_files

    filepath = '../languages/kinbank/' # the filepath where the kinbank files are kept

    k = get_kin_terms(filepath + file)

    print(file,k)

Nogai_noga1249.csv {'mG': 'karɨndas', 'mB': 'adanas', 'mZ': 'karɨndas', 'myB': 'ini', 'myZ': 'siŋli', 'mF': 'ata', 'mM': 'ana', 'mC': 'tuwgailar', 'mS': 'kede', 'mD': 'kɨz', 'mFF': 'atay', 'mFM': 'tetey', 'mMF': 'atay', 'mMM': 'tetey', 'mSS': 'yiyen', 'mSD': 'yiyеn kɨz', 'mDS': 'yiyen', 'mDD': 'yiyеn kɨz', 'mFB': 'akay', 'mFZ': 'abay', 'mMB': 'nagaš akay', 'mMZ': 'abay', 'mBS': 'yiyen', 'mBD': 'yiyеn kɨz', 'mZS': 'yiyen', 'mZD': 'yiyеn kɨz', 'mFZD': 'bölе', 'mFBD': 'bölе', 'mMBD': 'bölе', 'mMZD': 'bölе', 'mFBS': 'bölе', 'mFZS': 'bölе', 'mMBS': 'bölе', 'mMZS': 'bölе', 'mH': 'kiew', 'mW': 'pišе', 'mHF': 'kaynata', 'mHM': 'kaynana', 'mWF': 'kaynata', 'mWM': 'kaynana', 'mSW': 'kеšеk', 'mDH': 'kiеw', 'meB': 'adanas', 'meZ': 'karɨndas', 'mFeB': 'akay', 'mFyB': 'akay', 'mFeZ': 'abay', 'mFyZ': 'abay', 'mMeZ': 'abay', 'mMyZ': 'abay', 'mMeB': 'nagaš akay', 'mMyB': 'nagaš akay', 'meBS': 'yiyen', 'myBS': 'yiyen', 'meBD': 'yiyеn kɨz', 'myBD': 'yiyеn kɨz', 'meZS': 'yiyen', 'myZS': 'yiyen', 'meZD': '

Our random language is Nogai, a Turkic language spoken in Southeastern European Russia, Kazakhstan, Uzbekistan, Ukraine, Bulgaria, Romania and Turkey. But as we can see from printing the system, there's a lot of extra kin terms here that we don't need for our experiment today. We're only interested in Ego and Ego's parents' generations, but the system contains kin types like `mS` (male speaker's son) or `mDD` (make speaker's daughter's daughter). In the next section, we'll reduce `k` down to just the terms we're interested in.

### Condense the system down

Perhaps unintuitively, our first step will not be to filter by the kin type codes for the kinship terms in Ego and Ego's parents' generations. Instead, in `kintypes.py`, another file in this directory, I have written **a list of pairs of kin types**, where the first element in the pair is a parent type, and the second is their child; e.g. `mMeB` and `mMeBD` (mother's elder brother and mother's elder brother's daughter). We will be filtering our full kinship system `k` according to this list of pairs. There are two reasons for this:

1. Not every language has complete data in Kinbank. There may be some languages where a parent term is recorded and a corresponding child term is not. Incomplete data such as this will skew our probability distributions later, and cause problems with calculating MI. It's easier if we filter these out at this stage.
2. Since we're interested in whether kinship systems maintain patterns of terminological distinctions and mergers across these two generations, we will need to know which parent terms 'go with' which child terms.

The following function takes a kinship system as input, and outputs a list of tuples. The first element in the tuple is the parent term, the second is the corresponding child term.

In [8]:
def get_pairs(ks: dict) -> list:
    pairs_of_terms = []
    
    for pair in kt.ics_pairs:
        if pair[0] in ks and pair[1] in ks: # if both kin types are recorded in the kinship system,
            pairs_of_terms.append((ks[pair[0]],ks[pair[1]])) # then add the terms associated with those types to 
                                                             # pairs_of_terms
            
    return pairs_of_terms

But for our calculations, we'll still need to know which terms belong to which generation. Luckily, we know that the 0th element in each tuple is from Ego's parents' generation and the 1st element is from Ego's generation. So we can happily split these tuples down the middle and populate two lists with the terms.

In [14]:
def split_pairs(pairs: list) -> list:
    gn = []
    gn1 = []
    for pair in pairs:
        gn.append(pair[1])
        gn1.append(pair[0])
    
    return gn,gn1

To illustrate what these functions do, let's test them out with our random kinship system, `k`.

In [11]:
if testing:
    
    k_pairs = get_pairs(k)
    print(k_pairs)

[('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('n

In [15]:
if testing:
    
    k_gn,k_gn1 = split_pairs(k_pairs)

    print("Ego's generation: ", k_gn, '\n')

    print("Ego's parents' generation: ", k_gn1)

Ego's generation:  ['bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе', 'bölе'] 

Ego's parents' generation:  ['nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'nagaš akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'akay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', 'abay', '

`get_pairs()` gives us a long list of pairs, with lots of repetition. This is because some kin types will correspond to the same kin term in a language. For example, in English, mother's elder brother, mother's younger brother, father's elder brother and father's younger brother all correspond to the term 'uncle'. All of those people's children correspond to the word 'cousin'! So we'll end up with duplicate entries for these terms in some languages, but not all.

`split_pairs()` takes this long list of pairs and sorts it into terms that belong to Ego's generation and terms that belong to Ego's parents' generation. Importantly, since the order of the `pairs` list is preserved when we run `split_generations()`, we can still work out which terms form a parent-child pair by indexing `gn` and `gn1`.

Now we have our data structures, we can start to do some calculations.

### Calculating probabilities

To calculate entropy, we need a probability distribution over the terms in one single generation of a kinship system. So let's start with a function that can calculate the probability of a particular term.

Given a term and the full list of terms in the same generation, this function counts how many times that term exists in `generation` and divides that by the total length of `generation`.

In [17]:
def probability(term: str, generation: list) -> float:
    return generation.count(term)/len(generation)

So if we pick a term at random from our Nogai kinship system, it will output the probability of picking that term.

In [20]:
if testing:
    k_term = random.choice(k_gn1)
    print(k_term, probability(k_term,k_gn1))

nagaš akay 0.25


When calculating mutual information, we also need the **conditional entropy** of our system. To calculate this, we will need not only the probabilities of terms in a generation, but also the **joint probabilities** of every pair of terms across those two generations. In other words, we need to calculate the probabilities of our `pairs` output by `get_pairs`.

Given two terms, this function counts how many pairs made of those two terms exist in `pairs`, then divides that by the total length of `pairs`.

In [22]:
def joint_probability(term1: str, term2: str, pairs: list) -> float:
    pair = (term1,term2)
    return pairs.count(pair)/len(pairs)

Once again, we can test this with a random pair from our Nogai list of pairs:

In [25]:
if testing:
    k_pair = random.choice(k_pairs)
    print(k_pair, joint_probability(k_pair[0],k_pair[1],k_pairs))

('abay', 'bölе') 0.5


Now we can calculate probabilities, we can use these functions to calculate entropy, conditional entropy, and mutual information.

### Calculating entropy and mutual information

Entropy is defined as 

**INSERT EQUATION HERE LOL**

or, in English, it is the inverse sum over a distribution X of the probability of y * the log probability of y.

Entropy is a measure of the average level of uncertainty about the possible outcomes of a variable.

The functions we defined above only calculate a single probability at a time, so our next functions will need to iterate over the kinship system in order to have a full probability distribution. First, let's define a function that will iterate over a generation of the kinship system to output the entropy of that generation. 

Note: we only need one generation's entropy score to calculate mutual information - we will make the arbitrary choice to calculate the entropy of Ego's parents' generation later in this notebook.

In [40]:
def entropy(generation: list) -> list:
    entropy = 0
    for term in set(generation): # using a set as we want to count each unique term only once
        p = probability(term,generation)
        #print('entropy of',term,p*math.log(p))
        entropy += p*math.log(p)
    return -entropy

In [33]:
#if testing:
entropy(k_gn1)

entropy of nagaš akay -0.34657359027997264
entropy of akay -0.34657359027997264
entropy of abay -0.34657359027997264


1.0397207708399179

Moving on, conditional entropy is defined as

**ANOTHER EQUATION PLEASE**

or in English, the inverse sum over two distributions Y and X of the probability of each y * the log probability of each y given x.

Conditional entropy is the amount of information needed to describe the outcome of a random variable Y given that we already know the value of another random variable X.

To calculate it, we need the joint probability of each pair (given by `joint_probability()`) and the probability of one member of that pair (given by `probability()`). We can then calculate the conditional probability of parent term given child term as the joint probability of those terms over the probability of the parent term.

As before, we will define a function that iterates over all pairs to output the conditional entropy of Ego's generation given Ego's parents' generation.

In [34]:
def conditional_entropy(gn: list, gn1:list, pairs:list) -> float:
    entropy = 0
    for x,y in set(pairs): # x = parent, y = child
        p_xy = joint_probability(x,y,pairs)
        p_y = probability(y,g1)
        if p_xy > 0 and p_y > 0:
            #print('p(', x, '|', y,') = ', p_xy/p_y, 'p(y) = ', p_y)
            entropy += p_xy * math.log(p_xy/p_y)
    return -entropy

In [36]:
if testing:
    conditional_entropy(k_gn,k_gn1,k_pairs)

1.0397207708399179

Finally, mutual information is defined as

**LAST EQUATION**

or in English, entropy of X minus the conditional entropy of X given Y.

In this study, it is equal to the entropy of Ego's parents' generation minus the conditional entropy of Ego's parents' generation given Ego's generation. It tells us how much mutual dependence there is between these two generations; i.e. how much we can know about one by observing the other.

So long as we make sure to input the right entropy and conditional entropy values, we only need a simple function for this one:

In [38]:
def mutual_information(e,ce):
    return e - ce

In [50]:
if testing:
    mutual_information(entropy(k_gn1),conditional_entropy(k_gn,k_gn1,k_pairs))

0.0

Because entropy of Ego's parents' generation in Nogai and conditional entropy for Nogai are equal, the mutual information is 0. In other words, there is no mutual dependence between Ego's generation and Ego's parents' generation in Nogai.

And there we have it! Step 4 complete. We can now take any Kinbank file and output the mutual information between Ego's generation and Ego's parents' generation in that language.

But right at the beginning of this notebook, I mentioned using **simulations** to test the robustness of our claim that languages exhibit internal co-selection in their kinship systems. These simulations give us a baseline with which to compare the MI scores of real languages. Do languages across the world have greater mutual information between two generations than we would expect by chance?

## Simulations

If we want to argue that internal co-selection is a product of cultural evolution, we need to dispel the possibility that it occurs by chance.

To get an idea of what chance is, we need to create some randomly generated kinship systems. We can compare the MI of these simulations to the real languages to see whether the real languages have significantly greater mutual information between generations.

An important aspect of MI that we have not discussed so far: it is dependent on the amount of variation within the kinship system. A system with only one unique term in each generation would have MI of 0, which seems pretty terrible! But given this very limited variation (indeed, no variation), 0 is the highest MI such a language could have. As such, we perhaps need to modify our claim that kinship systems have "high MI" to be more specific: kinship systems in the wild have high MI *for the amount of variation in terminology they have*.

To compare real languages to simulations, we need a simulation which maintains the number of terms while randomising which child terms pair with which parent terms. To do this, we will take each language in our data, and randomly scramble which terms go with which types (within generation). This will randomise the syncretisms within the generations while maintaining the same amount of variation across the system overall.

To do this, we need to take the following steps:

1. Extract the kinship system of a language from kinbank (check!)
2. Filter the two generations we are interested in (check!)
3. Randomly reassign the kinship terms to new types.
4. Repeat the process a bunch of times for each language.

We already have the infrastructure for the first two! `get_kin_terms()`,  `get_pairs()` and `split_pairs()` will do this for us. So let's skip to 3, and write a function that randomises which terms form pairs, assuming that we have already extracted the kinship system and filtered the relevant pairs.

Remember that the order of `pairs` is preserved when we run `split_pairs()`. So when we pass `gn` and `gn1` to `shuffle_pairs()`, we know that we can re-unite our pairs by using the same index. Equally, when we shuffle `gn` and `gn1` in place, we know that we can safely combine them to make a new, randomised pair in place of the 'real' Nogai pair.

In [51]:
def shuffle_pairs(gn,gn1):
    random.shuffle(gn)
    random.shuffle(gn1)
    random_pairs = []
    for x,y in zip(gn,gn1):
        random_pairs.append((y,x))
    return random_pairs

In [56]:
if testing:
    sim_pairs = shuffle_pairs(k_gn,k_gn1)
    print(sim_pairs,k_pairs) # for comparison


[('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('akay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('akay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('akay', 'bölе'), ('akay', 'bölе'), ('abay', 'bölе'), ('akay', 'bölе'), ('abay', 'bölе'), ('abay', 'bölе'), ('akay', 'bölе'), ('nagaš akay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('nagaš akay', 'bölе'), ('abay', 'bölе'), ('a

Now we can treat `sim_pairs` just as we treated `pairs`! Let's calculate the entropy, conditional entropy, and mutual information of this simulated system.

In [57]:
if testing:
    sim_gn,sim_gn1 = split_pairs(sim_pairs)
    e = entropy(sim_gn1)
    ce = conditional_entropy(sim_gn,sim_gn1,sim_pairs)
    mi = mutual_information(e,ce)
    print(e,ce,mi)

1.0397207708399179 1.0397207708399179 0.0


Wait! That's exactly the same values as the real Nogai kinship system! I thought this was a randomised simulation - what gives?

Variation gives! Because Nogai only has one unique term in Ego's generation, its highest MI is 0, and the conditional entropy of Ego's parents' generation given Ego's generation is equal to entropy. Entropy remains the same regardless, because the amount of variation in the simulation **does not change** by design.

Let's pick a new random language that has more variation, so we can really see the difference.

In [72]:
if testing:
    random.seed(11)
    file2 = random.choice(all_kb_files)
    print(file2)
    k2 = get_kin_terms(filepath + file2)

    k2_pairs = get_pairs(k2)
    k2_gn,k2_gn1 = split_pairs(k2_pairs)
    k2_e = entropy(k2_gn1)
    k2_ce = conditional_entropy(k2_gn,k2_gn1,k2_pairs)
    k2_mi = mutual_information(k2_e, k2_ce)
    
    k2_sim = shuffle_pairs(k2_gn,k2_gn1)
    k2_sim_gn,k2_sim_gn1 = split_pairs(k2_sim)
    k2_sim_e = entropy(k2_sim_gn1)
    k2_sim_ce = conditional_entropy(k2_sim_gn,k2_sim_gn1,k2_sim)
    k2_sim_mi = mutual_information(k2_sim_e,k2_sim_ce)
    
    print('Sungwaloge:', k2_e,k2_ce,k2_mi)
    print('Simulation:',k2_sim_e,k2_sim_ce,k2_sim_mi)


Sungwaloge_(Nalemba_Simeone_Tari)_mari1426f.csv
Sungwaloge: 1.3862943611198906 0.6931471805599453 0.6931471805599453
Simulation: 1.3862943611198906 1.234244945579329 0.15204941554056162


Now we see that while the entropy of our new language, Sungwaloge, and its simulation are equal, the conditional entropy for the simulation is greater and therefore the mutual information of the simulation is lower. What about if we did this 1000 times? How often would the mutual information of the simulation be lower then?

## Tidying up

We have all the pieces we need now to calculate MI and simulate kinship systems - all we need to do is write a few more functions that stick all of those pieces together in a neat parcel.

First, a function that takes pairs and spits out entropy, conditional entropy, and MI:

In [81]:
def calculate_mi(pairs: list):
    gn,gn1 = split_pairs(pairs)
    e = entropy(gn1)
    ce = conditional_entropy(gn,gn1,pairs)
    mi = mutual_information(e,ce)
    return e,ce,mi

Second, a function that builds a simulation from scratch:

In [86]:
def simulate_ks(ks: dict) -> list:
    pairs = get_pairs(ks)
    gn,gn1 = split_pairs(pairs)
    simulation = shuffle_pairs(gn,gn1)
    return simulation

And a function that puts everything together, saves the results to a separate file, and outputs a `pandas` dataframe so that we can take a good look.

In [92]:
def ics_simulation(all_files:list, times):
    df = []
    
    for file in all_files:
        ks = get_kin_terms(filepath + file)
        language = file[:-13]
    
    
        for i in range(times):
            pairs = simulate_ks(ks)
            e,ce,mi = calculate_mi(pairs)
            mi2 = easy_mi(pairs)

            results = {}
            results['language'] = language
            results['simulation_number'] = str(i)
            results['mutual_information'] = mi
            results['CHECK mutual_information'] = mi2
            results['entropy'] = e
            results['conditional_entropy'] = ce

            df.append(results)
    
    #pd.DataFrame(df).to_csv('../data/raw/ics_sims.csv',index=False)
    
    return pd.DataFrame(df)

In [83]:
def ics_real(all_files):
    df = []
    
    for file in all_files:
        ks = get_kin_terms(filepath + file)
        language = file[:-13]
        pairs = get_pairs(ks)
        e,ce,mi = calculate_mi(pairs)
        
        results = {}
        results['language'] = language
        results['simulation_number'] = 'REAL'
        results['mutual_information'] = mi
        results['entropy'] = e
        results['conditional_entropy'] = ce

        df.append(results)
        
    #pd.DataFrame(df).to_csv('..data/raw/ics_real.csv',index=False)
    
    return pd.DataFrame(df)

In [84]:
ics_real(all_kb_files)

,language,simulation_number,mutual_information,entropy,conditional_entropy
0,'Are'are,REAL,0.693147,0.693147,-0.000000
1,Abui,REAL,0.000000,0.000000,0.000000
2,Achinese,REAL,0.000000,0.000000,0.000000
3,Achuar,REAL,0.257123,1.504788,1.247665
4,Aché,REAL,0.779452,0.974315,0.194863
...,...,...,...,...,...
1151,Zabana_(Kia),REAL,0.000000,0.000000,0.000000
1152,Zigula,REAL,0.000000,-0.000000,-0.000000
1153,Zinza,REAL,0.000000,0.000000,0.000000
1154,Zulu,REAL,1.039721,1.623862,0.584141


In [96]:
ics_simulation([file],10)

,language,simulation_number,mutual_information,CHECK mutual_information,entropy,conditional_entropy
0,Nogai,0,0.0,0.0,1.039721,1.039721
1,Nogai,1,0.0,0.0,1.039721,1.039721
2,Nogai,2,0.0,0.0,1.039721,1.039721
3,Nogai,3,0.0,0.0,1.039721,1.039721
4,Nogai,4,0.0,0.0,1.039721,1.039721
5,Nogai,5,0.0,0.0,1.039721,1.039721
6,Nogai,6,0.0,0.0,1.039721,1.039721
7,Nogai,7,0.0,0.0,1.039721,1.039721
8,Nogai,8,0.0,0.0,1.039721,1.039721
9,Nogai,9,0.0,0.0,1.039721,1.039721


In [94]:
# testing purposes only

from sklearn.metrics import mutual_info_score

def easy_mi(pairs):
        gn,gn1 = split_pairs(pairs)
        mi = mutual_info_score(gn,gn1)

        return mi

## TO DO

*remove languages where there's no data (they output 0 and may be conflated with actual 0 score languages)
*add commentary to final section
*run!!!!!!!!!!!!!!!!